DataFrame

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
from matplotlib import pyplot as plt 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df_results=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fifa/results.csv")


In [4]:
df_results = df_results.loc[df_results['date']>='2014-01-01'].loc[df_results['date']<'2022-12-31']
# 결측값있는 행 제거 
df_results.dropna(axis=0)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
36110,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True
36111,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True
36112,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False
36113,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False
36114,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False
...,...,...,...,...,...,...,...,...,...
44054,2022-09-27,Albania,Iceland,1.0,1.0,UEFA Nations League,Tirana,Albania,False
44055,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False
44056,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False
44057,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False


In [5]:
df_rankings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/fifa/fifa_ranking-2022-10-06.csv')
df_rankings.head()

,rank,country_full,country_abrv,total_points,previous_points,rank_change,confederation,rank_date
0,1,Germany,GER,57.0,0.0,0,UEFA,1992-12-31
1,96,Syria,SYR,11.0,0.0,0,AFC,1992-12-31
2,97,Burkina Faso,BFA,11.0,0.0,0,CAF,1992-12-31
3,99,Latvia,LVA,10.0,0.0,0,UEFA,1992-12-31
4,100,Burundi,BDI,10.0,0.0,0,CAF,1992-12-31


In [6]:
df_rankings = df_rankings.replace({"IR Iran":"Iran"})
df_rankings = df_rankings.loc[df_rankings['rank_date']>='2014-01-01'].loc[df_rankings['rank_date'] <='2022-12-31']

In [7]:
df_rankings = pd.DataFrame(df_rankings, columns = ['country_full', 'total_points', 'rank_date']).reset_index(drop=True)
df_rankings.head()

,country_full,total_points,rank_date
0,Cape Verde Islands,726.0,2014-01-16
1,Turkey,677.0,2014-01-16
2,Tunisia,689.0,2014-01-16
3,Honduras,692.0,2014-01-16
4,Peru,698.0,2014-01-16


In [8]:
df_rankings['rank_date'] = pd.to_datetime(df_rankings['rank_date'])
df_rankings['year'] = df_rankings['rank_date'].dt.year
df_rankings.head()

,country_full,total_points,rank_date,year
0,Cape Verde Islands,726.0,2014-01-16,2014
1,Turkey,677.0,2014-01-16,2014
2,Tunisia,689.0,2014-01-16,2014
3,Honduras,692.0,2014-01-16,2014
4,Peru,698.0,2014-01-16,2014


In [9]:
df_rankings.sort_values(by=['rank_date','year'],inplace=True, ascending=False)
latest_rankdate_index = df_rankings[['country_full', 'year']].drop_duplicates().index
df_rankings = df_rankings.loc[lambda x : x.index.isin(latest_rankdate_index)]

In [10]:
# result table, ranking table 을 year 기준으로 join 하기 
df_results['date'] = pd.to_datetime(df_results['date'])
df_results['year'] = df_results['date'].dt.year
df_results.reset_index(drop=True)


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year
0,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,2014
1,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,2014
2,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,2014
3,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,2014
4,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,2014
...,...,...,...,...,...,...,...,...,...,...
7945,2022-09-27,Norway,Serbia,0.0,2.0,UEFA Nations League,Oslo,Norway,False,2022
7946,2022-09-27,Sweden,Slovenia,1.0,1.0,UEFA Nations League,Stockholm,Sweden,False,2022
7947,2022-09-27,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,Pristina,Kosovo,False,2022
7948,2022-09-27,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,Athens,Greece,False,2022


In [11]:
nationality_names = df_results['home_team'].to_list() + df_results['away_team'].to_list() + df_rankings['country_full'].to_list() 
nationality_names = list(set(nationality_names))

In [ ]:
print(len(nationality_names))

303


In [11]:
dataset = df_results
dataset['home_team_total_points'] = np.nan
dataset['away_team_total_points'] = np.nan

In [12]:
for index in dataset.index:
  team1 = dataset.loc[index, ['home_team']]
  team2 = dataset.loc[index, ['away_team']]
  year = dataset.loc[index, ['year']]
  year = int(year)
  team1 = team1[0]
  team2 = team2[0]
  try:
    dataset.loc[index, ['home_team_total_points']] = int(df_rankings['total_points'].loc[df_rankings['country_full'] == team1].loc[df_rankings['year'] == year])
    dataset.loc[index, ['away_team_total_points']] = int(df_rankings['total_points'].loc[df_rankings['country_full'] == team2].loc[df_rankings['year'] == year])
  except:
    continue


player 14 ~ 23년도

In [13]:
# home team 
# 수비수 4명
for i in range(1, 5):
  col = 'home_B'+str(i)+'_point'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'home_M'+str(i)+'_point'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'home_T'+str(i)+'_point'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'home_K1_point'
dataset[col] = 0

In [14]:
# away team 
# 수비수 4명
for i in range(1, 5):
  col = 'away_B'+str(i)+'_point'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'away_M'+str(i)+'_point'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'away_T'+str(i)+'_point'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'away_K1_point'
dataset[col] = np.nan

In [15]:
# value 
# home team 
# 수비수 4명
for i in range(1, 5):
  col = 'home_B'+str(i)+'_value'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'home_M'+str(i)+'_value'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'home_T'+str(i)+'_value'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'home_K1_value'
dataset[col] = np.nan

In [16]:
# away team 
# 수비수 4명
for i in range(1, 5):
  col = 'away_B'+str(i)+'_value'
  dataset[col] = np.nan

# 미드필더 4명 
for i in range(1, 5):
  col = 'away_M'+str(i)+'_value'
  dataset[col] = np.nan

# 공격수 2명 
for i in range(1, 3):
  col = 'away_T'+str(i)+'_value'
  dataset[col] = np.nan

# 골키퍼 1명 
col = 'away_K1_value'
dataset[col] = np.nan

In [17]:
dataset.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,away_B2_value,away_B3_value,away_B4_value,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_T1_value,away_T2_value,away_K1_value
36110,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36111,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36112,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36113,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36114,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


더미데이터 뽑기 

In [ ]:
# 2022, 2023 fifa player example
df_player_22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/players/Career Mode player datasets - FIFA 22-23 example.xlsx", sheet_name=0)
df_player_23 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/players/Career Mode player datasets - FIFA 22-23 example.xlsx", sheet_name=1)

In [ ]:
df_player_22['year'] = 2021 # 2022 - 1
df_player_23['year'] = 2022 # 2023 - 1 
df_player = pd.concat([df_player_22, df_player_23], ignore_index=True)

In [ ]:
# 4:4:2 수비수 - 미드필더 - 공격수 
# nationality position 컬럼 사용 
# 수비수 : 미드필더 : ~~'M', 공격수 : 

In [ ]:
player_power = df_player['nation_position'].unique()
player_power 
# rcm , lcm -> cm 으로 바꿔주기 
# lcb, rcb -> cb 로 바꿔주기 
# lam, ram -> cam 
# rdm , ldm -> cdm

array(['RW', 'LW', 'ST', 'GK', 'LB', 'RCM', 'RCB', 'LCB', 'CDM', 'LCM',
       'RB', 'RDM', 'LDM', 'LAM', 'RAM', 'LF', 'RF', 'LM', 'CB', 'RM',
       'CM', 'CAM'], dtype=object)

In [ ]:
df_player['point'] = np.nan

In [ ]:
# for index in df_player.index:
#   key = df_player.loc[index, ['nation_position']][0]
#   key = str(key).lower()
#   if key == 'rcm' or key == 'lcm':
#     key = 'cm'
#   elif key == 'lcb' or key == 'rcb':
#     key = 'cb'
#   elif key == 'lam' or key == 'ram':
#     key = 'cam'
#   elif key == 'rdm' or key == 'ldm':
#     key = 'cdm'
#   df_player.loc[index, ['point']] = float(df_player.loc[index, [key]])

In [ ]:
for index in df_player.index:
  key = df_player.loc[index, ['nation_position']][0]
  key = str(key)
  if key == 'B': # 수비수 - lwb, rwb, lb, cb, rb
    data = max(df_player.loc[index, ['lwb', 'rwb', 'lb', 'cb', 'rb']])
  elif key == 'M': # 미드필더 - cam, lm, cm, rm, cdm, 
    data = max(df_player.loc[index, ['cam', 'lm', 'cm', 'rm','cdm']])
  elif key == 'W': # 공격수 - st, lw, rw, lf , cf, rf,
    data = max(df_player.loc[index, ['st', 'lw', 'rw', 'lf', 'cf', 'rf']])
  elif key == 'G': # 골키퍼 - gk 
    data = max(df_player.loc[index, ['gk']])
  df_player.loc[index, ['point']] = float(data)


NameError: ignored

In [ ]:
# 각 나라에서 해당 position이 몇위인지 ex) B1, B2, .. , M1, M2, .. 
df_player['rank_in_nation'] = df_player.groupby("nationality_name").groupby("nation_postion")['point'].rank(method="first", ascending=False)

AttributeError: ignored

In [ ]:
df_player.head()

,short_name,player_positions,overall,value_eur,wage_eur,age,height_cm,weight_kg,club_position,nationality_name,...,rm,lwb,cdm,rwb,lb,cb,rb,gk,year,point
0,L. Messi,"RW, ST, CF",93,78000000.0,320000.0,34,170,72,RW,Argentina,...,93,69,67,69,64,53,64,22,2021,92.0
1,Á. Di María,"RW, LW",87,49500000.0,160000.0,33,180,69,SUB,Argentina,...,87,73,71,73,70,61,70,21,2021,85.0
2,L. Martínez,ST,85,78000000.0,145000.0,23,174,72,LS,Argentina,...,83,68,69,68,66,65,66,22,2021,87.0
3,E. Martínez,GK,84,33500000.0,81000.0,28,195,88,GK,Argentina,...,41,36,40,36,36,35,36,85,2021,85.0
4,M. Acuña,"LB, LM",84,37000000.0,45000.0,29,172,69,LB,Argentina,...,84,84,84,84,84,81,84,22,2021,84.0


임의로 국가별로 groupby 한다음에 rank 붙이고 최종본 만들어보기 

In [ ]:
# 실험 데이터 
df_player_example = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/players/player FIFA 22-23 example.xlsx", sheet_name=0)

In [ ]:
for index in df_player_example.index:
  key = df_player_example.loc[index, ['nation_position']][0]
  key = str(key)
  if key == 'B': # 수비수 - lwb, rwb, lb, cb, rb
    data = max(df_player_example.loc[index, ['lwb', 'rwb', 'lb', 'cb', 'rb']])
  elif key == 'M': # 미드필더 - cam, lm, cm, rm, cdm, 
    data = max(df_player_example.loc[index, ['cam', 'lm', 'cm', 'rm','cdm']])
  elif key == 'W': # 공격수 - st, lw, rw, lf , cf, rf,
    data = max(df_player_example.loc[index, ['st', 'lw', 'rw', 'lf', 'cf', 'rf']])
  elif key == 'G': # 골키퍼 - gk 
    data = max(df_player_example.loc[index, ['gk']])
  df_player_example.loc[index, ['point']] = float(data)

In [ ]:
# 합치기 -> 실제로는 groupby 할때 year도 해줘야함 여기서는 22년도만 가져와서 임시로 테스트해봄 
df_player_example['rank_in_nation'] = df_player_example.groupby(['nationality_name','nation_position'])['point'].rank(ascending=False)
# df_player_example['year'] = 2021

In [ ]:
df_player_example['nation_position']

0     B
1     B
2     B
3     B
4     M
     ..
61    M
62    M
63    W
64    W
65    G
Name: nation_position, Length: 66, dtype: object

In [ ]:
#dataset_example = dataset.loc[dataset['year'] == 2021]
dataset_example = dataset
dataset_example.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,away_B2_value,away_B3_value,away_B4_value,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_W1_value,away_W2_value,away_G1_value
36110,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36111,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36112,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36113,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36114,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_player_example = df_player_example.astype({'rank_in_nation':'int'})
df_player_example.head()

,short_name,player_positions,overall,value_eur,wage_eur,age,height_cm,weight_kg,club_position,nationality_name,...,rm,lwb,cdm,rwb,lb,cb,rb,gk,point,rank_in_nation
0,L. Messi,"RW, ST, CF",93,78000000.0,320000.0,34,170,72,RW,Argentina,...,93,69,67,69,64,53,64,22,69.0,2
1,Á. Di María,"RW, LW",87,49500000.0,160000.0,33,180,69,SUB,Argentina,...,87,73,71,73,70,61,70,21,73.0,1
2,L. Martínez,ST,85,78000000.0,145000.0,23,174,72,LS,Argentina,...,83,68,69,68,66,65,66,22,68.0,3
3,E. Martínez,GK,84,33500000.0,81000.0,28,195,88,GK,Argentina,...,41,36,40,36,36,35,36,85,36.0,4
4,M. Acuña,"LB, LM",84,37000000.0,45000.0,29,172,69,LB,Argentina,...,84,84,84,84,84,81,84,22,84.0,1


In [ ]:
for index in dataset_example.index:
  team1 = dataset_example.loc[index, ['home_team']]
  team2 = dataset_example.loc[index, ['away_team']]
  
  year = dataset_example.loc[index, ['year']]
  year = int(year)
  team1 = team1[0]
  team2 = team2[0]
  # position B (수비수) 4명 
  for i in range(1, 5):
    col1 = 'home_B'+str(i)+'_point'
    col2 = 'home_B'+str(i)+'_value'
    col3 = 'away_B'+str(i)+'_point'
    col4 = 'away_B'+str(i)+'_point'
    try:
      data = df_player_example['point'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'B'].loc[df_player_example['nationality_name'] == team1]
      data2 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'B'].loc[df_player_example['nationality_name'] == team1]
      data3 = df_player_example['point'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'B'].loc[df_player_example['nationality_name'] == team2]
      data4 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'B'].loc[df_player_example['nationality_name'] == team2]
      dataset_example.loc[index, [col1]] = float(data)
      dataset_example.loc[index, [col2]] = float(data2)
      dataset_example.loc[index, [col3]] = float(data3)
      dataset_example.loc[index, [col4]] = float(data4)
    except:
      continue
  # position M (미드필더) 4명 
  for i in range(1, 5):
    col1 = 'home_M'+str(i)+'_point'
    col2 = 'home_M'+str(i)+'_value'
    col3 = 'away_M'+str(i)+'_point'
    col4 = 'away_M'+str(i)+'_value'
    try:
      data = df_player_example['point'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'M'].loc[df_player_example['nationality_name'] == team1]
      data2 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'M'].loc[df_player_example['nationality_name'] == team1]
      data3 = df_player_example['point'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'M'].loc[df_player_example['nationality_name'] == team2]
      data4 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'M'].loc[df_player_example['nationality_name'] == team2]
      dataset_example.loc[index, [col1]] = float(data)
      dataset_example.loc[index, [col2]] = float(data2)
      dataset_example.loc[index, [col3]] = float(data3)
      dataset_example.loc[index, [col4]] = float(data4)
    except:
      continue
  # position (공격수) 3명 
  for i in range(1, 4):
    col1 = 'home_W'+str(i)+'_point'
    col2 = 'home_W'+str(i)+'_value'
    col3 = 'away_W'+str(i)+'_point'
    col4 = 'away_W'+str(i)+'_value'
    try:
      data = df_player_example['point'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'W'].loc[df_player_example['nationality_name'] == team1]
      data2 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'W'].loc[df_player_example['nationality_name'] == team1]
      data3 = df_player_example['point'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'W'].loc[df_player_example['nationality_name'] == team2]
      data4 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == i].loc[df_player_example['nation_position'] == 'W'].loc[df_player_example['nationality_name'] == team2]
      dataset_example.loc[index, [col1]] = float(data)
      dataset_example.loc[index, [col2]] = float(data2)
      dataset_example.loc[index, [col3]] = float(data3)
      dataset_example.loc[index, [col4]] = float(data4)
    except:
      continue

  # # 골키퍼 
  col1 = 'home_G1_point'
  col2 = 'home_G1_value'
  col3 = 'away_G1_point'
  col4 = 'away_G1_value'
  try:
    data = df_player_example['point'].loc[df_player_example['rank_in_nation'] == 1].loc[df_player_example['nation_position'] == 'G'].loc[df_player_example['nationality_name'] == team1]
    data2 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == 1].loc[df_player_example['nation_position'] == 'G'].loc[df_player_example['nationality_name'] == team1]
    data3 = df_player_example['point'].loc[df_player_example['rank_in_nation'] == 1].loc[df_player_example['nation_position'] == 'G'].loc[df_player_example['nationality_name'] == team2]
    data4 = df_player_example['value_eur'].loc[df_player_example['rank_in_nation'] == 1].loc[df_player_example['nation_position'] == 'G'].loc[df_player_example['nationality_name'] == team2]
    dataset_example.loc[index, [col1]] = float(data)
    dataset_example.loc[index, [col2]] = float(data2)
    dataset_example.loc[index, [col3]] = float(data3)
    dataset_example.loc[index, [col4]] = float(data4)
  except:
    continue


In [ ]:
import random
# 결측값 채우기 
# 컬럼이 전체 null 이면 랜덤값 넣어주기 
#dataset_example['away_G1_point'] = dataset_example['away_G1_point'].fillna(random.randrange(50,90))

for i in range(1, 5):
  col = 'home_B'+str(i)+'_point'
  dataset_example[col] = dataset_example[col].fillna(int(random.randrange(40,90)))

# 미드필더 4명 
for i in range(1, 5):
  col = 'home_M'+str(i)+'_point'
  dataset_example[col] = dataset_example[col].fillna(int(random.randrange(40,90)))

# 공격수 2명 
for i in range(1, 3):
  col = 'home_W'+str(i)+'_point'
  dataset_example[col] = dataset_example[col].fillna(int(random.randrange(40,90)))

# 골키퍼 1명 
col = 'home_G1_point'
dataset_example[col] = dataset_example[col].fillna(int(random.randrange(40,90)))

In [ ]:
for i in range(1, 5):
  col = 'away_B'+str(i)+'_point'
  dataset_example[col] = dataset_example[col].fillna(random.randrange(50,90))

# 미드필더 4명 
for i in range(1, 5):
  col = 'away_M'+str(i)+'_point'
  dataset_example[col] = dataset_example[col].fillna(random.randrange(50,90))

# 공격수 2명 
for i in range(1, 3):
  col = 'away_W'+str(i)+'_point'
  dataset_example[col] = dataset_example[col].fillna(random.randrange(50,90))

# 골키퍼 1명 
col = 'away_G1_point'
dataset_example[col] = dataset_example[col].fillna(random.randrange(50,90))

In [ ]:
# 수비수 4명
for i in range(1, 5):
  col = 'home_B'+str(i)+'_value'
  dataset_example[col] = dataset_example[col].fillna(int(dataset_example[col].mean()))

# 미드필더 4명 
for i in range(1, 5):
  col = 'home_M'+str(i)+'_value'
  dataset_example[col] = dataset_example[col].fillna(int(dataset_example[col].mean()))

# 공격수 2명 
for i in range(1, 3):
  col = 'home_W'+str(i)+'_value'
  dataset_example[col] = dataset_example[col].fillna(int(dataset_example[col].mean()))

# 골키퍼 1명 
col = 'home_G1_value'
dataset_example[col] = dataset_example[col].fillna(int(dataset_example[col].mean()))

In [ ]:
# 수비수 4명
for i in range(1, 5):
  col = 'away_B'+str(i)+'_value'
  home_col = 'home_B'+str(i)+'_value'
  dataset_example[col] = dataset_example[col].fillna(int(dataset_example[home_col].mean()))

# 미드필더 4명 
for i in range(1, 5):
  col = 'away_M'+str(i)+'_value'
  home_col = 'home_M'+str(i)+'_value'
  dataset_example[col] = dataset_example[col].fillna(int(dataset_example[home_col].mean()))

# 공격수 2명 
for i in range(1, 3):
  col = 'away_W'+str(i)+'_value'
  home_col = 'home_W'+str(i)+'_value'
  dataset_example[col] = dataset_example[col].fillna(int(dataset_example[home_col].mean()))

# 골키퍼 1명 
col = 'away_G1_value'
home_col = 'home_G1_value'
dataset_example[col] = dataset_example[col].fillna(int(dataset_example[home_col].mean()))

In [ ]:
# year, date drop 
# dataset_example.drop('year', axis=1)
dataset_example.drop(['date', 'country', 'city', 'year'], axis=1)
# dataset_example.drop('country', axis=1)
# dataset_example.drop('city', axis=1)

In [ ]:
# 추출하기 
dataset_example.to_csv('dataset_example.csv')

최종 dataset 합치는 작업

In [18]:
# FIFA 15-23 player 최종본 
df_player14 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=0)
df_player15 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=1)
df_player16 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=2)
df_player17 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=3)
df_player18 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=4)
df_player19 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=5)
df_player20 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=6)
df_player21 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=7)
df_player22 = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/fifa/FIFA 15-23.xlsx", sheet_name=8)

In [19]:
df_player14['year'] = 2014
df_player15['year'] = 2015
df_player16['year'] = 2016
df_player17['year'] = 2017
df_player18['year'] = 2018
df_player19['year'] = 2019
df_player20['year'] = 2020
df_player21['year'] = 2021
df_player22['year'] = 2022

In [20]:
df_player = pd.concat([df_player14, df_player15, df_player16, df_player17, df_player18, df_player19, df_player20, df_player21, df_player22], ignore_index=True)

In [21]:
# nationality_name, Model_Position, year 로 group by 한 후, Position_Best_Rating 에 대해 rank 매기기 
df_player['rank_in_nation'] = df_player.groupby(['nationality_name','Model_Position', 'year'])['Position_Best_Rating'].rank(ascending=False)

In [22]:
df_player = df_player.astype({'rank_in_nation':'int'})
df_player.head()

,short_name,overall,value_eur,wage_eur,age,height_cm,weight_kg,nationality_name,preferred_foot,weak_foot,...,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,Position_Best_Rating,Model_Position,year,rank_in_nation
0,A. Meha,70,1600000,25000,28,174,76,Albania,Right,4,...,47,9,12,14,8,13,70,M,2014,3
1,E. Berisha,72,2500000,25000,25,194,92,Albania,Left,1,...,25,76,71,64,71,76,72,K,2014,1
2,E. Hysaj,67,500000,7000,20,182,70,Albania,Left,3,...,67,11,5,13,8,10,67,B,2014,3
3,E. Kace,69,825000,7000,20,161,58,Albania,Right,4,...,63,6,15,14,7,11,69,M,2014,4
4,J. Hyka,70,1800000,25000,26,169,62,Albania,Right,3,...,30,8,9,13,9,7,72,T,2014,2


In [23]:
df_player['Model_Position'].unique()

array(['M', 'K', 'B', 'T'], dtype=object)

In [24]:
dataset.drop(['date', 'country', 'city', 'year'], axis=1)

,home_team,away_team,home_score,away_score,tournament,neutral,home_team_total_points,away_team_total_points,home_B1_point,home_B2_point,...,away_B2_value,away_B3_value,away_B4_value,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_T1_value,away_T2_value,away_K1_value
36110,Kuwait,Jordan,1.0,2.0,WAFF Championship,True,246.0,393.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36111,Bahrain,Jordan,0.0,1.0,WAFF Championship,True,249.0,393.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36112,Namibia,Ghana,0.0,1.0,Friendly,False,295.0,714.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36113,Nigeria,Ethiopia,2.0,1.0,Friendly,False,656.0,290.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36114,Qatar,Kuwait,3.0,0.0,WAFF Championship,False,342.0,246.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44055,Norway,Serbia,0.0,2.0,UEFA Nations League,False,1473.0,1563.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44056,Sweden,Slovenia,1.0,1.0,UEFA Nations League,False,1553.0,1384.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44057,Kosovo,Cyprus,5.0,1.0,UEFA Nations League,False,1187.0,1183.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44058,Greece,Northern Ireland,3.0,1.0,UEFA Nations League,False,1437.0,1396.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df_player.loc[df_player['Model_Position'] == 'K'] # 734 rows

,short_name,overall,value_eur,wage_eur,age,height_cm,weight_kg,nationality_name,preferred_foot,weak_foot,...,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,Position_Best_Rating,Model_Position,year,rank_in_nation
1,E. Berisha,72,2500000,25000,25,194,92,Albania,Left,1,...,25,76,71,64,71,76,72,K,2014,1
18,R. M'Bohli,73,2500000,20000,28,188,83,Algeria,Right,3,...,25,77,64,66,72,81,73,K,2014,1
32,W. Caballero,79,6500000,90000,32,186,80,Argentina,Right,3,...,25,79,71,68,84,86,79,K,2014,1
39,M. Schwarzer,76,925000,45000,41,194,95,Australia,Right,3,...,25,73,78,66,79,75,76,K,2014,1
46,A. Manninger,72,800000,15000,37,189,85,Austria,Right,2,...,25,71,71,70,72,74,72,K,2014,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8029,Z. Steffen,77,9500000,73000,27,191,86,United States,Right,3,...,12,79,74,75,72,82,78,K,2022,1
8032,F. Muslera,79,1900000,34000,36,190,74,Uruguay,Right,2,...,16,76,80,72,82,77,79,K,2022,1
8042,J. Graterol,75,7000000,3000,25,176,82,Venezuela,Right,3,...,14,75,72,70,76,76,76,K,2022,1
8057,D. Ward,75,3800000,49000,29,191,88,Wales,Right,3,...,18,75,73,67,74,76,75,K,2022,1


In [ ]:
for index in dataset.index:
  team1 = dataset.loc[index, ['home_team']]
  team2 = dataset.loc[index, ['away_team']]
  
  year = dataset.loc[index, ['year']]
  year = int(year)
  team1 = team1[0]
  team2 = team2[0]
  # position B (수비수) 4명 
  for i in range(1, 5):
    col1 = 'home_B'+str(i)+'_point'
    col2 = 'home_B'+str(i)+'_value'
    col3 = 'away_B'+str(i)+'_point'
    col4 = 'away_B'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'B'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      data2 = df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'B'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      data3 = df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'B'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      data4 = df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'B'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col1]] = float(data)
      dataset.loc[index, [col2]] = float(data2)
      dataset.loc[index, [col3]] = float(data3)
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue
    
  # position M (미드필더) 4명 
  for i in range(1, 5):
    col1 = 'home_M'+str(i)+'_point'
    col2 = 'home_M'+str(i)+'_value'
    col3 = 'away_M'+str(i)+'_point'
    col4 = 'away_M'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      data2 = df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      data3 = df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      data4 = df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'M'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col1]] = float(data)
      dataset.loc[index, [col2]] = float(data2)
      dataset.loc[index, [col3]] = float(data3)
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue
  # position (공격수) 2명 -> T
  for i in range(1, 3):
    col1 = 'home_T'+str(i)+'_point'
    col2 = 'home_T'+str(i)+'_value'
    col3 = 'away_T'+str(i)+'_point'
    col4 = 'away_T'+str(i)+'_value'
    try:
      data = df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      data2 = df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
      data3 = df_player['Position_Best_Rating'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      data4 = df_player['value_eur'].loc[df_player['rank_in_nation'] == i].loc[df_player['Model_Position'] == 'T'].loc[df_player['nationality_name'] == team2].loc[df_player['year'] == year]
      dataset.loc[index, [col1]] = float(data)
      dataset.loc[index, [col2]] = float(data2)
      dataset.loc[index, [col3]] = float(data3)
      dataset.loc[index, [col4]] = float(data4)
    except:
      continue

  # # 골키퍼 
  col1 = 'home_K1_point'
  col2 = 'home_K1_value'
  col3 = 'away_K1_point'
  col4 = 'away_K1_value'
  try:
    data = df_player['Position_Best_Rating'].loc[df_player['Model_Position'] == 'K'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
    data2 = df_player['value_eur'].loc[df_player['Model_Position'] == 'K'].loc[df_player['nationality_name'] == team1].loc[df_player['year'] == year]
    data3 = df_player['Position_Best_Rating'].loc[df_player['Model_Position'] == 'K'].loc[df_player_example['nationality_name'] == team2].loc[df_player['year'] == year]
    data4 = df_player['value_eur'].loc[df_player['Model_Position'] == 'K'].loc[df_player_example['nationality_name'] == team2].loc[df_player['year'] == year]
    dataset.loc[index, [col1]] = float(data)
    dataset.loc[index, [col2]] = float(data2)
    dataset.loc[index, [col3]] = float(data3)
    dataset.loc[index, [col4]] = float(data4)
  except:
    continue

In [ ]:
dataset.head

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,year,...,away_B3_value,away_B4_value,away_M1_value,away_M2_value,away_M3_value,away_M4_value,away_T1_value,away_T2_value,away_T3_value,away_K1_value
36110,2014-01-01,Kuwait,Jordan,1.0,2.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36111,2014-01-04,Bahrain,Jordan,0.0,1.0,WAFF Championship,Doha,Qatar,True,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36112,2014-01-04,Namibia,Ghana,0.0,1.0,Friendly,Windhoek,Namibia,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36113,2014-01-04,Nigeria,Ethiopia,2.0,1.0,Friendly,Abuja,Nigeria,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36114,2014-01-04,Qatar,Kuwait,3.0,0.0,WAFF Championship,Doha,Qatar,False,2014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# 
nationality_list = dataset['home_team'].to_list() + dataset['away_team'].to_list()
nationality_list = list(set(nationality_list))
len(nationality_list)

283

In [ ]:
n_list = df_player['nationality_name'].to_list()
n_list = list(set(n_list))
len(n_list)

100

In [ ]:
dataset.to_csv('fifa_dataset.csv') # 최종 dataset 출력 